# Dash Development for MOT Metric Evalautions

This work aims to develope a Dashboard for MOT Metric Evaluation and Analytics.

## References used to support this development
- https://plotly.com/python/subplots/
- https://plotlydash.com/create-horizontal-bar-chart-with-slider-in-plotly-dash/
- https://plotly.com/python/horizontal-bar-charts/

In [2]:
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import random

In [3]:
app = JupyterDash(__name__)

In [4]:
df = pd.DataFrame(
    {
        "Trackers": ["Tracker_1", "Tracker_1", "Tracker_1"],
        "Datasets": ["MOT16", "MOT19", "MOT20"],
        "Metric": [1, 1, 1],
    }
)

fig0 = px.bar(df, x="Metric", y="Datasets", barmode="group", orientation="h", title="Datasets", text="Datasets")
fig0.update_traces(insidetextanchor='middle', textfont_color='white', showlegend=False)
fig0.update_xaxes(visible=False, showticklabels=False)
fig0.update_yaxes(visible=False, showticklabels=False)
fig0.show()

In [6]:
# Make data and make figure
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

metric_names = ["MOTA", "MOTP", "MODA", "CLR_Re", "CLR_Pr", "MTR", "PTR", "MLR"]
metric_plots = {}
for m in metric_names:
    metric_values = random.sample(range(1, 7), 6)
    df = pd.DataFrame(
        {
            "Trackers": ["Tracker_1", "Tracker_1", "Tracker_1", "Tracker_2", "Tracker_2", "Tracker_2"],
            "Datasets": ["MOT16", "MOT19", "MOT20", "MOT16", "MOT19", "MOT20"],
            "Metric": metric_values,
        }
    )
    fig = px.bar(df, x="Metric", y="Datasets", color="Trackers", barmode="group", orientation="h", title=m, text=[f"{row[1]} : {row[3]}" for row in df.itertuples()])
    fig.for_each_trace(lambda t: t.update(textfont_color='white', showlegend=False))
    # fig.update_yaxes(visible=False, showticklabels=False)
    metric_plots[m] = [fig["data"][trace] for trace in range(len(fig["data"]))]

For as many traces that exist per express figure, get the traces from each plot and store them in an array.

A Graph object contains a figure. Each figure has data and layout attributes. You can set the height in the layout.
```
dcc.Graph(
    id="my-graph",
    figure={
        "data": [
            {"x": [1, 2, 3], "y": [4, 1, 2], "type": "bar"},
            {"x": [1, 2, 3], "y": [2, 4, 5], "type": "bar"},
        ],
        "layout": {
            "title": "My Dash Graph",
            "height": 700,  # px
        },
    },
)
```
According to the Plotly figure object schema, height must be a number greater than or equal to 10, and its default is 450 (px).

Alternatively, you can change the viewport sizing in the parent container like:

`dcc.Graph(id='my-graph',style={'width': '90vh', 'height': '90vh'})`

In [22]:
fig = make_subplots(rows=1, cols=len(metric_names) + 1)

for trace in range(len(fig0["data"])):
    fig.append_trace(fig0["data"][trace], row=1, col=1)

for i, m in enumerate(metric_names):
    for trace in metric_plots[m]:
        fig.append_trace(trace, row=1, col=i+2)

fig.update_xaxes(visible=False, showticklabels=False)
fig.update_yaxes(visible=False, showticklabels=False)
final_graph = dcc.Graph(id="metrics" , figure=fig, style={'width': '150vh', 'height': '35vh'})

In [23]:
app.layout = html.Div(
    style={'backgroundColor': colors['background']},
    children=[
        html.H1(
            children='VMO MOT Metric Evaluations',
            style={
                'textAlign': 'center',
                'color': colors['text']
            }
        ),

        html.Div(
            children='MOT Performance Evaluations using HOTA, MOTA, CLEAR, and COUNT',
            style={
                'textAlign': 'center',
                'color': colors['text']
            }
        ),

        final_graph
    ]
)

In [25]:
app.run_server(mode='external', port=8053)
# app.run_server(mode='inline')

Dash app running on http://127.0.0.1:8053/
